https://devblogs.nvidia.com/deep-learning-self-driving-cars/

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import pickle
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import cv2

tf.python.control_flow_ops = tf

from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Cropping2D, Convolution2D

Using TensorFlow backend.


In [3]:
import bclone

In [4]:
DATA_DIR_STD = '../bclone-data-standard'
DATA_DIR_NEW = '../my_2018-02-22'

```bash
python preprocess-driving-log.py --src ../my_2018-02-22/driving_log_original.csv --dst ../my_2018-02-22/driving_log.csv
```

In [12]:
log_df = bclone.load_and_combine_logs(DATA_DIR_STD, DATA_DIR_NEW)
n_samples = len(log_df) * 3

print('Number of samples (including all three cameras): ', n_samples)

Number of samples (including all three cameras):  46218


In [10]:
log_df_train, log_df_valid = train_test_split(log_df, test_size=0.2)

log_df_train.shape, log_df_valid.shape

((12324, 7), (3082, 7))

In [16]:
def simple_model():

    model = Sequential()
    model.add(Lambda(lambda x: x / 255. - 0.5, input_shape=(160, 320, 3)))
    model.add(Flatten())
    model.add(Dense(1))
    model.compile(loss='mse', optimizer='adam')
    
    return model

def nvidia_model():
    
    model = Sequential()
    
    model.add( Lambda(lambda x: x / 255. - 0.5, input_shape=(160, 320, 3)) )
    model.add( Cropping2D(cropping=((70, 25), (0, 0))) )
    
    model.add( Convolution2D(24, 5, 5, subsample=(2, 2), activation='relu') )
    model.add( Convolution2D(36, 5, 5, subsample=(2, 2), activation='relu') )
    model.add( Convolution2D(48, 5, 5, subsample=(2, 2), activation='relu') )
    
    model.add( Convolution2D(64, 3, 3, subsample=(2, 2), activation='relu') )
    #model.add( Convolution2D(64, 3, 3, subsample=(2, 2), activation='relu') )

    model.add( Flatten() )

    model.add( Dense(100) )
    model.add( Dense(50) )
    model.add( Dense(10) )
    model.add( Dense(1) )
    
    model.compile(loss='mse', optimizer='adam')
    
    return model

In [18]:
model = nvidia_model()
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 160, 320, 3)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
cropping2d_1 (Cropping2D)        (None, 65, 320, 3)    0           lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 31, 158, 24)   1824        cropping2d_1[0][0]               
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 14, 77, 36)    21636       convolution2d_1[0][0]            
___________________________________________________________________________________________

In [24]:
train_gen = bclone.data_generator(log_df_train, batch_size=100, controls=['steering'])
valid_gen = bclone.data_generator(log_df_valid, batch_size=100, controls=['steering'])

history = bclone.fit_gen(model, train_gen, valid_gen, log_df_train, log_df_valid, n_epochs=2)

Epoch 1/2
36972/36972 [==============================] - 1501s - loss: 0.0272 - val_loss: 0.0247
Epoch 2/2
36972/36972 [==============================] - 1450s - loss: 0.0264 - val_loss: 0.0246


In [25]:
#model.save('model2.h5')